In [23]:
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import os
from synphot import SpectralElement, Empirical1D, SourceSpectrum, Observation
from scipy.interpolate import RegularGridInterpolator
from scipy.optimize import minimize, LinearConstraint

SPEC_PIPE_LOC = "{}/JPL/W12_Drops/spec_paper/Spec_pipeline".format(os.path.expanduser("~"))
sys.path.append(SPEC_PIPE_LOC)
os.environ['SPEC_PIPE_LOC'] = SPEC_PIPE_LOC
from Spec_pipeline import SDSS_Spec


sys.path.append("Gas_and_dust/fullspec_dust_models/")
from polWave import PolWave

from polWave_gas import PolWave_gas

In [37]:
data = np.loadtxt("pol_measurements.dat", usecols=[1,2])
p_measured = data[:,0]
p_unc = data[:,1]

In [3]:
#Read the observed spectrum
spec = SDSS_Spec("W0116-0505", 3.173, "spec-7046-56568-0425.fits")

In [4]:
#Load the bands.
def load_bands():
    #Load the filters.
    R_spec = np.loadtxt("M_SPECIAL_R.txt")
    I_bess = np.loadtxt("M_BESS_I.txt")
    v_high = np.loadtxt("v_HIGH.txt", skiprows=2)

    R_spec = R_spec[R_spec[:,1]>0.01]
    I_bess = I_bess[I_bess[:,1]>0.01]
    v_high = v_high[v_high[:,1]>0.01]

    #Transform the wavelengths to angstroms.
    R_spec[:,0] *= 10
    I_bess[:,0] *= 10
    v_high[:,0] *= 10

    v_high = v_high[(v_high[:,0]>4000.) & (v_high[:,0]<8000.)]
    I_bess = I_bess[(I_bess[:,0]>4000.) & (I_bess[:,0]<8000.)]
    R_spec = R_spec[(R_spec[:,0]>4000.) & (R_spec[:,0]<8000.)]

    Rbp= SpectralElement(Empirical1D, points=R_spec[:,0], lookup_table=R_spec[:,1]/100., keep_neg=False)
    Ibp= SpectralElement(Empirical1D, points=I_bess[:,0], lookup_table=I_bess[:,1]/100., keep_neg=False)
    vbp= SpectralElement(Empirical1D, points=v_high[:,0], lookup_table=v_high[:,1]/100., keep_neg=False)

    return [vbp, Rbp, Ibp]

In [5]:
bands = load_bands()

In [6]:
full_spec = SourceSpectrum(Empirical1D, points=spec.lam_obs, lookup_table=spec.flam, keep_neg=True)

In [43]:
def get_p_bb(dust_type, spec, bands, fw=True, bw=True):

    theta_angles = np.arange(0., 90.1, 5.)
    psi_angles = np.arange(0., 90.1, 5.)

    p_bb = np.zeros((len(bands), len(theta_angles), len(psi_angles)))
    p_bb_interp = list()

    if dust_type=="gas":
        model = PolWave_gas(fw=fw,bw=bw)
        for iband in range(len(bands)):
            p_bb_interp.append(model.p)
        #     for jtheta, theta in enumerate(theta_angles):
        #         th_aux = theta*np.ones(len(psi_angles))
        #         p_bb[iband, jtheta] = model.p((th_aux, psi_angles))

    else:
        model = PolWave(dust_type, folder="Gas_and_dust/fullspec_dust_models/", fw=fw, bw=bw)

        for iband, band in enumerate(bands):
            obs_I = Observation(full_spec, band)
            Ibb = obs_I.effstim(flux_unit='flam').value
            for jtheta, theta in enumerate(theta_angles):
                for kpsi, psi in enumerate(psi_angles):
                    th_aux = theta*np.ones(len(spec.lam_obs))
                    psi_aux = psi*np.ones(len(spec.lam_obs))
                    p_lam = model.p((spec.lam_rest.to(u.AA).value, th_aux, psi_aux))
            
                    Q_spec = SourceSpectrum(Empirical1D, points=spec.lam_obs, lookup_table=spec.flam * p_lam, keep_neg=True)

                    obs_Q = Observation(Q_spec, band)
                    Qbb = obs_Q.effstim(flux_unit='flam').value

                    p_bb[iband, jtheta, kpsi] = Qbb/Ibb
            p_bb_interp.append(RegularGridInterpolator((theta_angles, psi_angles), p_bb[iband]))
    return p_bb_interp
    

In [46]:
def chi2(x, p_bb_interp, bands, pm, pu):
    if x[0]<0 or x[0]>90. or x[1]<0. or x[1]>90.:
        return np.inf
    p_bb = np.zeros(len(bands))
    for iband in range(len(bands)):
        p_bb[iband] = p_bb_interp[iband](([x[0]], [x[1]]))
    return np.sum(((p_bb-pm)/pu)**2)
    

In [47]:
dust_types = ["gas","SMC","LMC","MW"]
fw = True
bw = True

for dust_type in dust_types:
    print(dust_type)
    p_bb_interp = get_p_bb(dust_type,spec,bands,fw=fw,bw=bw)

    #Set the linear constraints.
    x0 = np.array([45., 45.])
    G = np.identity(x0.shape[0])
    min_vals = [0.,0.]
    max_vals = [90., 90.]
    lincon = LinearConstraint(G, min_vals, max_vals)

    #Run the fit
    xopt = minimize(chi2, x0=x0, constraints=lincon, args=(p_bb_interp, bands, p_measured, p_unc)) 
    print(xopt)       

gas
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 88.99310886372244
       x: [ 4.021e+01  5.915e+01]
     nit: 8
     jac: [ 5.492e-03 -4.338e-03]
    nfev: 26
    njev: 8
SMC
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1189.7161469744003
       x: [ 6.000e+01  5.017e+01]
     nit: 16
     jac: [-6.816e-01  1.450e-03]
    nfev: 60
    njev: 16
LMC
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1201.423501878542
       x: [ 4.889e+01  3.717e+01]
     nit: 8
     jac: [ 4.120e-04 -3.052e-04]
    nfev: 26
    njev: 8
MW
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 719.1457483631455
       x: [ 3.000e+01  1.612e-14]
     nit: 45
     jac: [-4.959e-01  2.812e+00]
    nfev: 200
    njev: 45


In [50]:
dust_types = ["gas","SMC","LMC","MW"]
fw = True
bw = False

for dust_type in dust_types:
    print(dust_type)
    p_bb_interp = get_p_bb(dust_type,spec,bands,fw=fw,bw=bw)

    #Set the linear constraints.
    x0 = np.array([45., 45.])
    G = np.identity(x0.shape[0])
    min_vals = [0.,0.]
    max_vals = [90., 90.]
    lincon = LinearConstraint(G, min_vals, max_vals)

    #Run the fit
    xopt = minimize(chi2, x0=x0, constraints=lincon, args=(p_bb_interp, bands, p_measured, p_unc)) 
    print(xopt)       

gas
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 88.99310896442009
       x: [ 3.701e+01  5.447e+01]
     nit: 7
     jac: [ 2.117e-03 -1.230e-03]
    nfev: 23
    njev: 7
SMC
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1098.973825659164
       x: [ 4.958e+01  3.500e+01]
     nit: 29
     jac: [ 8.423e-02  3.023e-02]
    nfev: 166
    njev: 28
LMC
 message: Inequality constraints incompatible
 success: False
  status: 4
     fun: 1262.541972480868
       x: [ 9.000e+01  6.943e+01]
     nit: 7
     jac: [       inf  5.482e+01]
    nfev: 24
    njev: 7
MW
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 524.1002597024407
       x: [ 3.118e+01  4.368e-14]
     nit: 20
     jac: [ 8.736e-03  1.320e+00]
    nfev: 72
    njev: 20


In [54]:
dust_types = ["gas","SMC","LMC","MW"]
fw = False
bw = True

for dust_type in dust_types:
    print(dust_type)
    p_bb_interp = get_p_bb(dust_type,spec,bands,fw=fw,bw=bw)

    #Set the linear constraints.
    x0 = np.array([45., 45.])
    G = np.identity(x0.shape[0])
    min_vals = [0.,0.]
    max_vals = [90., 90.]
    lincon = LinearConstraint(G, min_vals, max_vals)

    #Run the fit
    xopt = minimize(chi2, x0=x0, constraints=lincon, args=(p_bb_interp, bands, p_measured, p_unc))
    print(xopt)
    for iband in range(len(bands)):
        print(p_measured[iband], p_unc[iband], p_bb_interp[iband](([xopt.x[0]], [xopt.x[1]])))
    print()

gas
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 88.99310896442009
       x: [ 3.701e+01  5.447e+01]
     nit: 7
     jac: [ 2.117e-03 -1.230e-03]
    nfev: 23
    njev: 7
0.0973 0.0037 [0.11300468]
0.1085 0.0023 [0.11300468]
0.1466 0.0041 [0.11300468]

SMC
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 30.201927054480105
       x: [ 2.691e+01  1.054e-14]
     nit: 19
     jac: [ 3.242e-05  2.791e-01]
    nfev: 65
    njev: 19
0.0973 0.0037 [0.11566744]
0.1085 0.0023 [0.1080789]
0.1466 0.0041 [0.13696248]

LMC
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 23.69514134746996
       x: [ 2.641e+01  1.540e-14]
     nit: 15
     jac: [ 5.245e-06  1.022e+00]
    nfev: 51
    njev: 15
0.0973 0.0037 [0.10545929]
0.1085 0.0023 [0.09864263]
0.1466 0.0041 [0.14939274]

MW
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 44.2592391758398
       x: [ 2